In [103]:
import pprint
import json

In [125]:
#its a test sample, i extract 10 values randomly from the output of sql code(), if it work then i will extract the 5 most popular movies

def get_random_movies():
    sql_query = """
        WITH avg_vote AS (
            SELECT AVG(m.vote_average) AS avg_vote
            FROM movies AS m
        ),
        avg_votes_cast AS (
            SELECT
                AVG(m.vote_count) AS avg_count,
                MAX(m.vote_count) AS max_count
            FROM movies AS m
        ),
        avg_vote_score AS (
            SELECT
                m.id,
                avc.max_count,
                avc.avg_count,
                CASE
                    WHEN m.vote_average >= (av.avg_vote * 1.5) THEN 'High'
                    WHEN m.vote_average >= (av.avg_vote * 0.5) THEN 'Medium'
                    ELSE 'Low'
                END AS score
            FROM movies AS m
            CROSS JOIN avg_vote AS av
            CROSS JOIN avg_votes_cast AS avc
        )
        SELECT 
            m.title AS movie, 
            avs.score AS viewer_score_classification
        FROM movies AS m
        JOIN avg_vote_score avs ON m.id = avs.id
        ORDER BY RANDOM()
        LIMIT 10;
    """

    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()

        cur.execute(sql_query)
        rows = cur.fetchall()

        movie_list = [{"title": row[0], "classification": row[1]} for row in rows]

        cur.close()
        conn.close()

        return movie_list

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        return []

if __name__ == "__main__":
    random_movies = get_random_movies()
    pprint.pprint(random_movies, sort_dicts=False)


[{'title': "Through the Attic's Eye", 'classification': 'Low'},
 {'title': 'Cracka Ho on da Down Lo', 'classification': 'Low'},
 {'title': 'Bi Jocks And Big Cocks 2', 'classification': 'Low'},
 {'title': 'Lesbian Western', 'classification': 'Medium'},
 {'title': 'Amityville Poltergeist', 'classification': 'Medium'},
 {'title': 'Iron Maiden: Flight 666', 'classification': 'High'},
 {'title': 'UFC on FX 4: Maynard vs. Guida', 'classification': 'High'},
 {'title': "I'll Remember You as You Were, Not as What You'll Become",
  'classification': 'High'},
 {'title': 'Little Turtle', 'classification': 'Low'},
 {'title': 'Dimensión Abierta', 'classification': 'Low'}]


In [129]:
def get_movies_with_cast():
    sql_query = """
        WITH avg_vote AS (
            SELECT AVG(m.vote_average) AS avg_vote
            FROM movies AS m
        ),
        avg_votes_cast AS (
            SELECT
                AVG(m.vote_count) AS avg_count,
                MAX(m.vote_count) AS max_count
            FROM movies AS m
        ),
        avg_vote_score AS (
            SELECT
                m.id,
                avc.max_count,
                avc.avg_count,
                CASE
                    WHEN m.vote_average >= (av.avg_vote * 1.5) THEN 'High'
                    WHEN m.vote_average >= (av.avg_vote * 0.5) THEN 'Medium'
                    ELSE 'Low'
                END AS score,
                CASE
                    WHEN LN(NULLIF(m.vote_count, 0)) >= LN(NULLIF(avc.avg_count, 0) * 10) THEN 'High'
                    WHEN LN(NULLIF(m.vote_count, 0)) >= LN(NULLIF(avc.avg_count, 0) * 2) THEN 'Medium'
                    ELSE 'Low'
                END AS vote_count
            FROM movies AS m
            CROSS JOIN avg_vote AS av
            CROSS JOIN avg_votes_cast AS avc
        ),
        cast_ranking AS (
            SELECT
                am.movie_id,
                a.name,
                a.popularity,
                ROW_NUMBER() OVER (PARTITION BY am.movie_id ORDER BY a.popularity DESC) AS ranking
            FROM actor_movies AS am
            JOIN actors a ON am.actor_id = a.id
        ),
        cast_list AS (
            SELECT
                cr.movie_id,
                ARRAY_AGG(cr.name ORDER BY cr.popularity DESC) FILTER ( WHERE ranking <= 5 ) AS movie_cast
            FROM cast_ranking AS cr
            GROUP BY cr.movie_id
        )
        SELECT
            m.title AS Movie,
            avs.score AS viewer_score_classification,
            m.vote_average,
            avs.vote_count AS vote_classification,
            ln(NULLIF(m.vote_count, 0)) AS vote_count,
            cl.movie_cast,
            m.overview
        FROM movies AS m
        JOIN avg_vote_score avs ON m.id = avs.id
        JOIN cast_list cl ON m.id = cl.movie_id
        WHERE EXISTS (
            SELECT 1
            FROM JSONB_ARRAY_ELEMENTS(m.genres) AS genre
            WHERE (genre ->> 'id')::INTEGER = 28 -- Action genre
        )
        GROUP BY cl.movie_cast, m.id, avs.score, m.vote_average, avs.vote_count, m.vote_count, m.title
        ORDER BY
            (avs.vote_count = 'High') DESC,
            CASE
                WHEN avs.vote_count = 'High' THEN m.vote_average
                ELSE m.vote_count
            END DESC,
            m.vote_count DESC
        LIMIT 5;
    """

    try:
        params = config()  
        conn = psycopg2.connect(**params)
        cur = conn.cursor()

        cur.execute(sql_query)
        rows = cur.fetchall()

        columns = ["movie", "viewer_score_classification", "vote_average", "vote_classification", "vote_count", "movie_cast","summary"]

        df = pd.DataFrame(rows, columns=columns)
        cur.close()
        conn.close()

        return df

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error:", error)
        return pd.DataFrame()

if __name__ == "__main__":
    movies_df = get_movies_with_cast()
    pprint.pprint(movies_df, sort_dicts=False)

                                               movie  \
0                                    The Dark Knight   
1      The Lord of the Rings: The Return of the King   
2                                      Seven Samurai   
3                                          Inception   
4  The Lord of the Rings: The Fellowship of the Ring   

  viewer_score_classification vote_average vote_classification  vote_count  \
0                        High          8.5                High   10.417119   
1                        High          8.5                High   10.114720   
2                        High          8.5                High    8.241967   
3                        High          8.4                High   10.520240   
4                        High          8.4                High   10.150855   

                                          movie_cast  \
0  [Gary Oldman, 陳冠希, Cillian Murphy, Morgan Free...   
1  [Cate Blanchett, Viggo Mortensen, Elijah Wood,...   
2                     [三船敏

In [130]:
movies_df.to_csv("movies.csv", index=False, encoding="utf-8-sig")